# How to use the pytxtfilter.py

This notebook aims to provide a simple introduction to the `pytxtfilter.py` module. This module provides tools to filter delimited text files **while** reading to reduce the memory requirements. Typical usage includes reading a very big table of which a relatively small subset of the data will need to be kept for further analysis. This module was developed the ebird EBD files in mind, but is flexible and can be applied to any kind of delimited text file.

## Simple usage

In [1]:
import pytxtfilter as tf

The main class of this module is `DelimTxt` which is a wrapper to the `csv` module. In addition to `csv.reader`'s arguments it has one positional and one named arguments: `name` and `has_header`. All other arguments are passed to `csv.reader`. Main difference is that `DelimTxt` does not expect a filename/handler. It first collects necessary information about the format of the files and filtering them. For a simple csv file with header:

In [2]:
quakes = tf.DelimTxt("quakes dataset", has_header=True)

Filters are then defined on columns they will be applied. We will be working with files formatted similary to the `quakes` dataset of R - more info [https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/quakes.html]:

    [,1] lat       numeric	Latitude of event
    [,2] long      numeric	Longitude
    [,3] depth     numeric	Depth (km)
    [,4] mag       numeric	Richter Magnitude
    [,5] stations  numeric	Number of stations reporting


Let's build a filter on magnitude, "mag", column: 

In [3]:
magnitude_filter = quakes.create_filter_template("magnitude", "mag", float)
magnitude_filter.add_comparison(">=", 4.8)

This will create a filter, "magnitude", which will act on the "mag" column and expects a `float`. We also created a comparison criteria, equal to or greater than 4.8.

In [4]:
magnitude_filter.add_comparison("<")

The seconod comparison we added did not have a value - it only defined 'smaller than'. The value will need to be given when the filter is used.

Filters are only templates at first. To use a filter, `use_filter` needs to be called, and any missing comparison values need to be given. To use the "magnitude" to select only values that are >= 4.8 and < 5.5, we need:

In [5]:
quakes.use_filter("magnitude", 5.5)

In [6]:
quakes.print_filter("magnitude")

Filter 'magnitude':
[1]: * >= 4.8
[2]: * < 5.5
On column: 'mag'


Let's create a second filter, on "depth":

In [7]:
depth_filter = quakes.create_filter_template("depth", "depth", int)
depth_filter.add_comparison(">=")
depth_filter.add_comparison("<=")

This time we have not supplied any values so both need to be supplied when the filter is used:

In [8]:
quakes.use_filter("depth", 60, 150)

In [9]:
quakes.print_filters()

== Filters in use ==
1. Filter 'magnitude':
[1]: * >= 4.8
[2]: * < 5.5
On column: 'mag'

2. Filter 'depth':
[1]: * >= 60
[2]: * <= 150
On column: 'depth'



Finally, we can use our filters on a file by calling the `.process` method:

In [10]:
quakes.process("../../data/notebook/quakes.csv")

,lat,long,depth,mag,stations
7,-11.7,166.1,82,4.8,43
50,-18.97,185.25,129,5.1,73
81,-34.02,180.21,75,5.2,65
91,-15.48,167.53,128,5.1,61
110,-36.95,177.81,146,5,35
121,-14.65,166.97,82,4.8,28
129,-27.72,181.7,94,4.8,59
139,-25.2,182.6,149,4.9,31
159,-20.9,169.84,93,4.9,31
214,-20.47,185.68,93,5.4,85
223,-22.43,184.48,65,4.9,48
226,-13.65,166.66,71,4.9,52
230,-22.34,171.52,106,5,43
244,-18.2,183.68,107,4.8,52
254,-22.5,170.4,106,4.9,38
266,-26.17,184.2,65,4.9,37
292,-13.34,166.2,67,4.8,18
296,-16.4,185.86,148,5,47
321,-14.82,167.32,123,4.8,28
325,-12.49,166.36,74,4.9,55
330,-11.41,166.24,83,5.3,55
331,-19.1,183.87,61,5.3,42
338,-27.19,182.18,69,5.4,68
342,-27.38,181.7,80,4.8,13
357,-22.19,171.4,150,5.1,49
378,-15.36,186.66,112,5.1,57
383,-25.5,182.82,124,5,25
412,-22.24,184.56,99,4.8,57
413,-20.07,169.14,66,4.8,37
443,-17.97,168.52,146,4.8,33
445,-22.55,183.81,82,5.1,68
474,-10.79,166.06,142,5,40
477,-37.93,177.47,65,5.4,65
486,-23.58,183.4,94,5.2,79
501,-16.51,187.1,62,4.9,46
547,-11.54

## Use case: ebird EBD format

ebird's EBD files are large tab delimited TSV files. Strings are not quoted. We can define an ebird object as following:

In [11]:
import csv
import pytxtfilter as tf

ebird = tf.DelimTxt("ebird", has_header=True, dialect="unix",
                    delimiter="\t", quoting=csv.QUOTE_NONE, escapechar='\\')

Typically, we would need to filter the data against a set of bird species. Let's define a filter for this purpose:

In [12]:
species = ebird.create_filter_template("species", "SCIENTIFIC NAME", str)
species.add_comparison("in")

The `in` operant will let us use a list of `str` as the comparison value and any row will pass if the "SCIENTIFIC NAME" is **in** this list.

Another commonly used filter 'breeding'. Here a NULL or "F" value is considered as "no breeding code". A simple comparison would be **not in** against \[NULL, F\]. 'Not in' operation is not defined - we can define any operation using the `.define_operant` method:

In [13]:
breeding = ebird.create_filter_template("breeding", "BREEDING BIRD ATLAS CODE", str)
breeding.define_operant("!in", lambda x, y: x not in y)
breeding.add_comparison("!in", ["", "F"])

Let's use these two filters:

In [14]:
ebird.use_filter("species", ["Merops apiaster", "Halcyon smyrnensis"])
#                 ["Vanellus spinosus", "Periparus ater",
#                  "Tachybaptus ruficollis", "Sturnus vulgaris"])
ebird.use_filter("breeding")

In [15]:
ebird.process("../../data/ebd-test/ebd_TR_sample.txt")

GLOBAL UNIQUE IDENTIFIER	LAST EDITED DATE	TAXONOMIC ORDER	CATEGORY	COMMON NAME	SCIENTIFIC NAME	SUBSPECIES COMMON NAME	SUBSPECIES SCIENTIFIC NAME	OBSERVATION COUNT	BREEDING BIRD ATLAS CODE	BREEDING BIRD ATLAS CATEGORY	AGE/SEX	COUNTRY	COUNTRY CODE	STATE	STATE CODE	COUNTY	COUNTY CODE	IBA CODE	BCR CODE	USFWS CODE	ATLAS BLOCK	LOCALITY	LOCALITY ID	LOCALITY TYPE	LATITUDE	LONGITUDE	OBSERVATION DATE	TIME OBSERVATIONS STARTED	OBSERVER ID	SAMPLING EVENT IDENTIFIER	PROTOCOL TYPE	PROTOCOL CODE	PROJECT CODE	DURATION MINUTES	EFFORT DISTANCE KM	EFFORT AREA HA	NUMBER OBSERVERS	ALL SPECIES REPORTED	GROUP IDENTIFIER	HAS MEDIA	APPROVED	REVIEWED	REASON	TRIP COMMENTS	SPECIES COMMENTS	
URN:CornellLabOfOrnithology:EBIRD:OBS673195471	2018-11-04 09:20:42	9474	species	White-throated Kingfisher	Halcyon smyrnensis			2	P 	C3		Turkey	TR	Osmaniye	TR-80							Kirmitli Kus Cenneti--genel area	L2435177	H	37.1812449	36.1239910	2016-04-19	07:00:00	obsr1135534	S49665584	Traveling	P22	EBIRD	240	10.000		8	1	G3501844	0	1	0			

Let's create a third filter on "date" column, which is the 28th column:

In [16]:
date = ebird.create_filter_template("date", 28, str)
date.add_comparison(">=")
date.add_comparison("<=")

In [17]:
ebird.use_filter("date", "2017-03-01", "2018-03-01")
ebird.process("../../data/ebd-test/ebd_TR_sample.txt")

GLOBAL UNIQUE IDENTIFIER	LAST EDITED DATE	TAXONOMIC ORDER	CATEGORY	COMMON NAME	SCIENTIFIC NAME	SUBSPECIES COMMON NAME	SUBSPECIES SCIENTIFIC NAME	OBSERVATION COUNT	BREEDING BIRD ATLAS CODE	BREEDING BIRD ATLAS CATEGORY	AGE/SEX	COUNTRY	COUNTRY CODE	STATE	STATE CODE	COUNTY	COUNTY CODE	IBA CODE	BCR CODE	USFWS CODE	ATLAS BLOCK	LOCALITY	LOCALITY ID	LOCALITY TYPE	LATITUDE	LONGITUDE	OBSERVATION DATE	TIME OBSERVATIONS STARTED	OBSERVER ID	SAMPLING EVENT IDENTIFIER	PROTOCOL TYPE	PROTOCOL CODE	PROJECT CODE	DURATION MINUTES	EFFORT DISTANCE KM	EFFORT AREA HA	NUMBER OBSERVERS	ALL SPECIES REPORTED	GROUP IDENTIFIER	HAS MEDIA	APPROVED	REVIEWED	REASON	TRIP COMMENTS	SPECIES COMMENTS	
URN:CornellLabOfOrnithology:EBIRD:OBS514832232	2017-06-29 08:54:31	9474	species	White-throated Kingfisher	Halcyon smyrnensis			1	F 	C1		Turkey	TR	Osmaniye	TR-80							TR - Osmaniye - Kesmeburun - 37.1638 x 36.1835	L5777125	P	37.1638090	36.1835150	2017-04-30	13:00:00	obsr758505	S37864950	Stationary	P21	EBIRD_TUR	60			6	1	G23812

In [18]:
ebird.print_filters()

== Filters in use ==
1. Filter 'species':
[1]: * in ['Merops apiaster', 'Halcyon smyrnensis']
On column: 'SCIENTIFIC NAME'

2. Filter 'breeding':
[1]: * !in ['', 'F']
On column: 'BREEDING BIRD ATLAS CODE'

3. Filter 'date':
[1]: * >= 2017-03-01
[2]: * <= 2018-03-01
On column: 28

